In [ ]:
import copy
Q = int(input())
A = []
for _ in range(Q):
    A.append(list(map(int,input().split())))
B = copy.deepcopy(A)
B = sorted(B,key=lambda x:x[1])
D = {}
cnt = 1
for i in range(Q):
    if (B[i][0]==1 or B[i][0]==3) and B[i][1] not in D:
        D[B[i][1]] = cnt
        cnt += 1
invD = {v:k for (k,v) in D.items()}
k = 0
while 2**k<len(D):
    k += 1
M = 2**k
T = [0 for _ in range(M+1)]
def update(i,x):
    while i<=M:
        T[i] += x
        i += i&(-i)
def cumsum(i):
    cnt = 0
    while i>0:
        cnt += T[i]
        i -= i&(-i)
    return cnt
for i in range(Q):
    if A[i][0]==1:
        update(D[A[i][1]],1)
    elif A[i][0]==2:
        update(D[A[i][1]],-1)
    else:
        x = D[A[i][1]]
        low = x-1
        high = M+1
        while high-low>1:
            mid = (high+low)//2
            if cumsum(mid)>cumsum(x-1):
                high = mid
            else:
                low = mid
        if high>M:
            print(-1)
        else:
            print(invD[high])

In [ ]:
# 最大フロー問題(Ford-Fulkerson法)
n,m,s,t = map(int,input().split())
cap = [[0 for _ in range(n+1)] for _ in range(n+1)]  # 各有向辺の要領
for _ in range(m):
    a,b,c = map(int,input().split())
    cap[a][b] = c
def push_flow(v,t1,f):  # vからt1にフローfを流す
    if v==t1:return f
    visit[v] = True
    for u in range(1,n+1):  # vの次に訪れる頂点uを全探索
        if visit[u]==False and cap[v][u]>0:
            f1 = push_flow(u,t1,min(f,cap[v][u]))
            if f1>0:
                cap[v][u] -= f1  # 順辺の容量を減らす
                cap[u][v] += f1  # 逆辺の容量を増やす
                return f1
    return 0
ans = 0
while True:
    visit = [False]*(n+1)
    f = push_flow(s,t,100)
    if f==0:break
    ans += f
print(ans)

In [ ]:
# セグメント木（１点更新、区間取得）
# 区間最小値
INFTY = (1<<31)-1
N,Q = map(int,input().split())
k = 0
while 2**k<N:
    k += 1
K = 2**k  # ノード数
T = [INFTY]*(2*K)
def update(i,x):  # 更新
    i += K
    T[i] = x
    i = i//2
    while i>0:
        T[i] = min(T[2*i],T[2*i+1])
        i //= 2
def find(l,r):  # 区間最小値
    l += K  # 左側の頂点
    r += K  # 右側の頂点
    res = INFTY
    while l<r:
        if l&1:  # 右子の場合
            res = min(res,T[l])
            l += 1  # 右隣の頂点に移る
        if r&1:  # &0としたいところだが、この操作は意味ない
            r -= 1  # 左隣の頂点に移る
            res = min(res,T[r])
        l >>= 1  # 親に移る
        r >>= 1  # 親に移る
    return res
for _ in range(Q):
    q,x,y = map(int,input().split())
    if q==0:
        update(x,y)
    else:
        y += 1
        print(find(x,y))

In [ ]:
# セグメント木（区間更新、点取得）:Range Add Query, BITでも実装可能
N,Q = map(int,input().split())
K = 0
while 2**K<N:
    K += 1
M = 2**K  # ノード数
T = [0]*(2*M)
def update(l,r,z):  # 区間更新：[l,r)にzを加算
    l += M-1  # 頂点番号(1-origin)
    r += M-1  # 頂点番号(1-origin)
    while l<r:
        if l&1:  # 右子の場合
            T[l] += z
            l += 1
        if r&1:  # 右子の場合
            r -= 1  # 左隣の頂点に移る
            T[r] += z
        l >>= 1  # 親に移る
        r >>= 1  # 親に移る
def find(l):  # １点取得
    x = l+M-1  # 頂点番号(1-origin)
    cnt = 0
    while x>0:
        cnt += T[x]
        x = x//2
    return cnt
for _ in range(Q):
    qry = list(map(int,input().split()))
    if qry[0]==0:
        s,t,x = qry[1:]
        t += 1
        update(s,t,x)
    else:
        print(find(qry[1]))

In [ ]:
# セグメント木（点更新、区間和）:RSQ, BITでも実装可能
N,Q = map(int,input().split())
K = 0
while 2**K<N:
    K += 1
M = 2**K  # ノード数
T = [0]*(2*M)
R = [0]*(2*M)
for i in range(M,2*M):
    R[i] = (i-M+1,i-M+2)
for i in range(M-1,0,-1):
    l1,r1 = R[2*i]
    l2,r2 = R[2*i+1]
    R[i] = (l1,r2)
def update(i,x):  # １点更新
    i += M-1  # 頂点番号(1-origin)
    T[i] += x
    i >>= 1  # 親に移る
    while i>0:
        T[i] = T[2*i]+T[2*i+1]
        i >>= 1  # 親に移る
def find(l,r):  # 区間取得
    l += M-1  # 頂点番号(1-origin)
    r += M-1  # 頂点番号(1-origin)
    result = 0
    while l<r:
        if l&1:  # lが奇数の場合（右の子）
            result += T[l]
            l += 1  # 右隣に移動
        if r&1:  # rが奇数の場合（右の子）
            r -= 1  # 左隣に移動
            result += T[r]
        l >>= 1  # 親に移動
        r >>= 1  # 親に移動
    return result

for _ in range(Q):
    qry = list(map(int,input().split()))
    if qry[0]==0:
        i,x = qry[1:]
        update(i,x)
    else:
        s,t = qry[1:]
        t += 1
        print(find(s,t))

In [ ]:
# セグメント木（区間更新、区間最小値取得）
N,Q = map(int,input().split())
K = 0
while 2**K<N:
    K += 1
M = 2**K  # ノード数
INFTY = (1<<31)-1
T1 = [INFTY]*(2*M)
T2 = [INFTY]*(2*M)
def ref(x):  # T2の値をT1に反映してT1の値を返す
    if T2[x]<INFTY:
        T1[x] = T2[x]
        if x<M:
            T2[2*x] = T2[x]
            T2[2*x+1] = T2[x]
        T2[x] = INFTY
    return T1[x]
def eval(x):  # 遅延評価
    x += M  # 頂点番号(0-origin)
    for h in range(K,0,-1):
        i = x>>h  # 祖先の頂点番号
        ref(i)
def fix(x):  # T1の値を葉から根に向かって確定させる
    x += M  # 頂点番号(0-origin)
    x >>= 1  # 親に移る
    while x>0:
        T1[x] = min(ref(2*x),ref(2*x+1))
        x >>= 1  # 親に移る
def update(l,r,z):  # 区間更新
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l1= l+M  # 頂点番号(0-origin)
    r1= r+M  # 頂点番号(0-origin)
    while l1<r1:
        if l1&1:  # 右子の場合
            T2[l1] = z
            l1 += 1
        if r1&1:  # &0としたいところだが、この操作は意味ない
            r1 -= 1  # 左隣の頂点に移る
            T2[r1] = z
        l1 >>= 1  # 親に移る
        r1 >>= 1  # 親に移る
    eval(l)  # 左側の頂点の遅延評価
    fix(l)  # T1の値を葉から根に向かって確定させる
    eval(r-1)  # 右側の頂点の遅延評価
    fix(r-1)  # T1の値を葉から根に向かって確定させる 
def find(l,r):  # 区間取得
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l += M  # 頂点番号(0-origin)
    r += M  # 頂点番号(0-origin)
    resl = INFTY
    resr = INFTY
    while l<r:
        if l&1:
            resl = min(resl,ref(l))
            l += 1
        if r&1:
            r -= 1
            resr = min(resr,ref(r))
        l >>= 1
        r >>= 1
    return min(resl,resr)
for _ in range(Q):
    qry = list(map(int,input().split()))
    if qry[0]==0:
        s,t,x = qry[1:]
        t += 1
        update(s,t,x)
    else:
        s,t = qry[1:]
        t += 1
        print(find(s,t))

In [ ]:
# セグメント木（区間加算、区間和）:RAQ,RSQ
N,Q = map(int,input().split())
K = 0
while 2**K<N:
    K += 1
M = 2**K  # ノード数
T1 = [0]*(2*M)
T2 = [0]*(2*M)
R = [0]*(2*M)
for i in range(M,2*M):
    R[i] = (i-M+1,i-M+2)
for i in range(M-1,0,-1):
    l1,r1 = R[2*i]
    l2,r2 = R[2*i+1]
    R[i] = (l1,r2)
def ref(x):  # T2の値をT1に反映してT1の値を返す
    if T2[x]>0:
        T1[x] += T2[x]*(R[x][1]-R[x][0])
        if x<M:
            T2[2*x] += T2[x]
            T2[2*x+1] += T2[x]
        T2[x] = 0
    return T1[x]
def eval(x):  # 遅延評価
    x += M-1  # 頂点番号(1-origin)
    for h in range(K,0,-1):
        i = x>>h  # 祖先の頂点番号
        ref(i)
def fix(x):  # T1の値を葉から根に向かって確定させる
    x += M-1  # 頂点番号(1-origin)
    x >>= 1  # 親に移る
    while x>0:
        T1[x] = ref(2*x)+ref(2*x+1)
        x >>= 1  # 親に移る
def update(l,r,z):  # 区間更新
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l1= l+M-1  # 頂点番号(1-origin)
    r1= r+M-1  # 頂点番号(1-origin)
    while l1<r1:
        if l1&1:  # 右子の場合
            T2[l1] += z
            l1 += 1
        if r1&1:  # &0としたいところだが、この操作は意味ない
            r1 -= 1  # 左隣の頂点に移る
            T2[r1] += z
        l1 >>= 1  # 親に移る
        r1 >>= 1  # 親に移る
    eval(l)  # 左側の頂点の遅延評価
    fix(l)  # T1の値を葉から根に向かって確定させる
    eval(r-1)  # 右側の頂点の遅延評価
    fix(r-1)  # T1の値を葉から根に向かって確定させる 
def find(l,r):  # 区間取得
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l += M-1  # 頂点番号(1-origin)
    r += M-1  # 頂点番号(1-origin)
    resl = 0
    resr = 0
    while l<r:
        if l&1:
            resl += ref(l)
            l += 1
        if r&1:
            r -= 1
            resr += ref(r)
        l >>= 1
        r >>= 1
    return resl+resr
for _ in range(Q):
    qry = list(map(int,input().split()))
    if qry[0]==0:
        s,t,x = qry[1:]
        t += 1
        update(s,t,x)
    else:
        s,t = qry[1:]
        t += 1
        print(find(s,t))

In [ ]:
# セグメント木（区間加算、区間最小）:RAQ,RMQ
N,Q = map(int,input().split())
K = 0
while 2**K<N:
    K += 1
M = 2**K  # ノード数
T1 = [0]*(2*M)
T2 = [0]*(2*M)
R = [0]*(2*M)
for i in range(M,2*M):
    R[i] = (i-M+1,i-M+2)
for i in range(M-1,0,-1):
    l1,r1 = R[2*i]
    l2,r2 = R[2*i+1]
    R[i] = (l1,r2)
def ref(x):  # T2の値をT1に反映してT1の値を返す
    if T2[x]!=0:
        T1[x] += T2[x]
        if x<M:
            T2[2*x] += T2[x]
            T2[2*x+1] += T2[x]
        T2[x] = 0
    return T1[x]
def eval(x):  # 遅延評価
    x += M  # 頂点番号(0-origin)
    for h in range(K,0,-1):
        i = x>>h  # 祖先の頂点番号
        ref(i)
def fix(x):  # T1の値を葉から根に向かって確定させる
    x += M  # 頂点番号(0-origin)
    x >>= 1  # 親に移る
    while x>0:
        T1[x] = min(ref(2*x),ref(2*x+1))
        x >>= 1  # 親に移る
def update(l,r,z):  # 区間更新
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l1= l+M  # 頂点番号(0-origin)
    r1= r+M  # 頂点番号(0-origin)
    while l1<r1:
        if l1&1:  # 右子の場合
            T2[l1] += z
            l1 += 1
        if r1&1:  # &0としたいところだが、この操作は意味ない
            r1 -= 1  # 左隣の頂点に移る
            T2[r1] += z
        l1 >>= 1  # 親に移る
        r1 >>= 1  # 親に移る
    eval(l)  # 左側の頂点の遅延評価
    fix(l)  # T1の値を葉から根に向かって確定させる
    eval(r-1)  # 右側の頂点の遅延評価
    fix(r-1)  # T1の値を葉から根に向かって確定させる 
def find(l,r):  # 区間取得
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l += M  # 頂点番号(0-origin)
    r += M  # 頂点番号(0-origin)
    resl = 1<<31
    resr = 1<<31
    while l<r:
        if l&1:
            resl = min(resl,ref(l))
            l += 1
        if r&1:
            r -= 1
            resr = min(resr,ref(r))
        l >>= 1
        r >>= 1
    return min(resl,resr)
for _ in range(Q):
    qry = list(map(int,input().split()))
    if qry[0]==0:
        s,t,x = qry[1:]
        t += 1
        update(s,t,x)
    else:
        s,t = qry[1:]
        t += 1
        print(find(s,t))

In [ ]:
# mergesort
def merge(A,left,mid,right):  # 半開区間
    L = A[left:mid]  # ソート済み
    L.append(10**9)  # 番兵の追加
    R = A[mid:right]  # ソート済み
    R.append(10**9)  # 番兵の追加
    i = 0  # Lのカーソル
    j = 0  # Rのカーソル
    for k in range(left,right):  # k番目に何を入れるか
        if L[i]<=R[j]:  # 大小比較（大小の定義は別にあるものとする）
            A[k] = L[i]  # k番目はLから
            i += 1  # Lのカーソルを更新
        else:
            A[k] = R[j]  # k番目はRから
            j += 1  # Rのカーソルを更新
def mergesort(A,left,right):
    if left+1<right:
        mid = (left+right)//2
        mergesort(A,left,mid)
        mergesort(A,mid,right)
        merge(A,left,mid,right)
N = int(input())
A = list(map(int,input().split()))
mergesort(A,0,len(A))  # indexは0-origin
print(*A)

In [ ]:
# 平面走査
N = int(input())
A = [list(map(int,input().split())) for _ in range(N)]
X = set()
Y = set()
for i in range(N):
    x1,y1,x2,y2 = A[i]
    X.add(x1)
    X.add(x2)
    Y.add(y1)
    Y.add(y2)
X = sorted(list(X))
Y = sorted(list(Y))
Dx = {X[i]:i+1 for i in range(len(X))}
Dy = {Y[i]:i+1 for i in range(len(Y))}
B = []
for i in range(N):
    x1,y1,x2,y2 = A[i]
    B.append((Dx[x1],Dy[y1],Dx[x2],Dy[y2]))
Bx = []
By = []
for i in range(N):
    x1,y1,x2,y2 = B[i]
    if y1==y2:
        if x1>x2:
            x1,x2 = x2,x1
        Bx.append((x1,y1,0))
        Bx.append((x2,y2,2))
    else:
        if y1>y2:
            y1,y2 = y2,y1
        Bx.append((x1,y1,1))
        Bx.append((x2,y2,3))
        By.append((x1,y1,x2,y2))
Bx = sorted(Bx,key=lambda x:x[1])
Bx = sorted(Bx,key=lambda x:x[2])
Bx = sorted(Bx,key=lambda x:x[0])
By = sorted(By,key=lambda x:x[1])
By = sorted(By,key=lambda x:x[0])
k = 0
while 2**k<2*N:
    k += 1
M = 2**k
T = [0]*(M+1)
def update(i,x):
    while i<=M:
        T[i] += x
        i += i&(-i)
def cumsum(i):
    tot = 0
    while i>0:
        tot += T[i]
        i -= i&(-i)
    return tot
cnt = 0
ans = 0
for i in range(len(Bx)):
    x,y,d = Bx[i]
    if d==0:
        update(y,1)
    elif d==1:
        x1,y1,x2,y2 = By[cnt]
        ans += cumsum(y2)-cumsum(y1-1)
        cnt += 1
    elif d==2:
        update(y,-1)
print(ans)

In [ ]:
# 尺取り法
N,K = map(int,input().split())
A = list(map(int,input().split()))
r = -1
cnt = 0
tot = 0
for l in range(N):
    if l>0:
        tot -= A[l-1]
    while r<N and tot<=K:
        r += 1
        if r<N:
            tot += A[r]
    cnt += r-l
print(cnt)

In [ ]:
#0-1BFS
from collections import deque
H,W = map(int,input().split())
rs,cs = map(int,input().split())
rs -= 1
cs -= 1
rt,ct = map(int,input().split())
rt -= 1
ct -= 1
S = [input().strip() for _ in range(H)]
INFTY = 10**6+10
A = [[INFTY for _ in range(W)] for _ in range(H)]
A[rs][cs] = 0
que = deque([(rs,cs,0,"L"),(rs,cs,0,"R"),(rs,cs,0,"U"),(rs,cs,0,"D")])
D = {"L":(0,-1),"R":(0,1),"U":(-1,0),"D":(1,0)}
while que:
    i,j,c,d = que.popleft()
    for e in D:
        di,dj = D[e]
        if d==e and 0<=i+di<H and 0<=j+dj<W and S[i+di][j+dj]=="." and A[i+di][j+dj]>=c:
            A[i+di][j+dj] = c
            que.appendleft((i+di,j+dj,c,d))
        elif d!=e and 0<=i+di<H and 0<=j+dj<W and S[i+di][j+dj]=="." and A[i+di][j+dj]>=c+1:
            A[i+di][j+dj] = c+1
            que.append((i+di,j+dj,c+1,e))
print(A[rt][ct])

In [ ]:
# datetimeオブジェクト
import datetime as dt
S = list(map(int,input().split("-")))
T = list(map(int,input().split("-")))
dt0 = dt.date(2021,12,31)
dt1 = dt.date(S[0],S[1],S[2])
dt2 = dt.date(T[0],T[1],T[2])
d1 = (dt1-dt0).days-1
q1 = d1//7
r1 = d1%7
d2 = (dt2-dt0).days
q2 = d2//7
r2 = d2%7
n1 = q1*2+min(2,r1)
n2 = q2*2+min(2,r2)
print(n2-n1)

In [ ]:
# 固定区間最大値（ダブリング）
from collections import deque
N = int(input())
P = [0,0]+list(map(int,input().split()))
G = {i:[] for i in range(1,N+1)}
for i in range(2,N+1):
    G[P[i]].append(i)  # 有向木
# 根からの距離
dist = [-1]*(N+1)
dist[1] = 0
que = deque([(1,0)])
while que:
    x,d = que.popleft()
    for y in G[x]:
        dist[y] = d+1
        que.append((y,d+1))
K = 0
while 2**K<N:
    K += 1
dp = [[0 for _ in range(N+1)] for _ in range(K+1)]
for i in range(2,N+1):
    dp[0][i] = P[i]
for k in range(1,K+1):
    for i in range(2,N+1):
        if (1<<k) > dist[i]:
            dp[k][i] = 0
        else:
            dp[k][i] = dp[k-1][dp[k-1][i]]
Q = int(input())
for _ in range(Q):
    x,y = map(int,input().split())
    d = dist[y]-dist[x]-1
    ans = y
    for k in range(K):
        if d>>k & 1:
            ans = dp[k][ans]
    print(ans)

In [ ]:
#trie（未完成）
N,M,Q = map(int,input().split())
trie = {0:["0",{"end":-1}]}  # 頂点番号:[頂点文字,{次の頂点に向かう辺＝次の頂点文字:次の頂点番号}]
# 単語の追加
cnt = 1  # 頂点番号
for _ in range(N):
    S = input().strip()
    cur = 0
    for i in range(len(S)):
        a = S[i]
        if a not in trie[cur][1]:
            trie[cur][1][a] = cnt
            trie[cnt] = [a,{}]
            if i==len(S)-1:
                trie[cnt][1]["end"] = -1  # 単語の終わりを示す
            cnt += 1
        else:
            if i==len(S)-1:
                trie[trie[cur][1][a]][1]["end"] = -1
        cur = trie[cur][1][a]
# 単語の削除
for _ in range(M):
    T = input().strip()
    cur = 0  # 頂点の初期値（トライの根）
    Pa = {}  # 親頂点
    for i in range(len(T)):
        a = T[i]
        if a in trie[cur][1]:
            Pa[trie[cur][1][a]] = cur
            cur = trie[cur][1][a]
            if i==len(T)-1:
                if "end" in trie[cur][1]:
                    # 単語の削除
                    del trie[cur][1]["end"]
                    if len(trie[cur][1])>0:break
                    while "end" not in trie[cur][1]:
                        b = trie[cur][0]
                        del trie[cur]
                        cur = Pa[cur]
                    del trie[cur][1][b]
                else:break
        else:break
# 単語の有無チェック
for _ in range(Q):
    W = input().strip()
    cur = 0
    for i in range(len(W)):
        a = W[i]
        if a in trie[cur][1]:
            cur = trie[cur][1][a]
            if i==len(W)-1:
                if "end" in trie[cur][1]:
                    print("Yes")
                else:
                    print("No")
        else:
            print("No")
            break

In [ ]:
# heap: push
N = int(input())
A = list(map(int,input().split()))
A.append(int(input()))
# 入れ替え
cur = N
while cur>0 and A[(cur-1)//2]>A[cur]:
    A[(cur-1)//2],A[cur] = A[cur],A[(cur-1)//2]
    cur = (cur-1)//2
print(*A)

.pyファイルから.ipynbファイルへの変換

In [ ]:
# heap木
N,Q = map(int,input().split())
A = list(map(int,input().split()))
for _ in range(Q):
    qry = list(input().split())
    if len(qry)==2:
        x = int(qry[1])
        A.append(x)
        cur = len(A)-1
        while cur>0 and A[(cur-1)//2]>A[cur]:
            A[(cur-1)//2],A[cur] = A[cur],A[(cur-1)//2]
            cur = (cur-1)//2
    else:
        print(A[0])
        x = A.pop()
        if len(A)==0:
            continue
        A[0] = x
        cur = 0
        while 2*cur+(len(A)%2)+1<len(A):
            l = A[2*cur+1]
            if 2*cur+2<len(A):
                r = A[2*cur+2]
            else:
                r = 10**10
            amin = min(l,r)
            if A[cur]<=amin:break
            if l==amin:
                A[cur],A[2*cur+1] = A[2*cur+1],A[cur]
                cur = 2*cur+1
            else:
                A[cur],A[2*cur+2] = A[2*cur+2],A[cur]
                cur = 2*cur+2
print(*A)

In [ ]:
# セグメント木（区間更新、区間和取得）
N,Q = map(int,input().split())
K = 0
while 2**K<N:
    K += 1
M = 2**K  # ノード数
T1 = [0]*(2*M)
T2 = [0]*(2*M)
def ref(x):  # T2の値をT1に反映してT1の値を返す
    k = 0
    while x*(1<<k)<2*M:
        k += 1
    k -= 1
    if T2[x]!=0:
        T1[x] = T2[x]*(1<<k)  # 区間和の計算
        if x<M:
            T2[2*x] = T2[x]
            T2[2*x+1] = T2[x]
        T2[x] = 0
    return T1[x]
def eval(x):  # 遅延評価（ｘから見て親の遅延評価を根から順番に行う）
    x += M-1  # 頂点番号(1-origin)
    for h in range(K,0,-1):
        i = x>>h  # 祖先の頂点番号
        ref(i)
def fix(x):  # T1の値を葉から根に向かって確定させる
    x += M-1  # 頂点番号(1-origin)
    x >>= 1  # 親に移る
    while x>0:
        T1[x] = ref(2*x)+ref(2*x+1)  # 区間和の計算
        x >>= 1  # 親に移る
def update(l,r,z):  # 区間更新：[l,r)をzに変更
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l1= l+M-1  # 頂点番号(1-origin)
    r1= r+M-1  # 頂点番号(1-origin)
    while l1<r1:
        if l1&1:  # 右子の場合
            T2[l1] = z
            l1 += 1
        if r1&1:  # 右子の場合
            r1 -= 1  # 左隣の頂点に移る
            T2[r1] = z
        l1 >>= 1  # 親に移る
        r1 >>= 1  # 親に移る
    eval(l)  # 左側の頂点の遅延評価
    fix(l)  # T1の値を葉から根に向かって確定させる
    eval(r-1)  # 右側の頂点の遅延評価
    fix(r-1)  # T1の値を葉から根に向かって確定させる 
def find(l,r):  # 区間取得
    eval(l)  # 左側の頂点の遅延評価
    eval(r-1)  # 右側の頂点の遅延評価
    l += M-1  # 頂点番号(1-origin)
    r += M-1  # 頂点番号(1-origin)
    resl = 0
    resr = 0
    while l<r:
        if l&1:  # 右子の場合
            resl += ref(l)
            l += 1
        if r&1:  # 右子の場合
            r -= 1
            resr += ref(r)
        l >>= 1
        r >>= 1
    return resl+resr
for _ in range(Q):
    qry = list(map(int,input().split()))
    if qry[0]==0:
        s,t,x = qry[1:]
        t += 1
        update(s,t,x)
    else:
        s,t = qry[1:]
        t += 1
        print(find(s,t))

In [ ]:
N,M,A,B = map(int,input().split())
T = [[[] for _ in range(B)] for _ in range(A)]
X = list(map(int,input().split()))
Y = list(map(int,input().split()))
for i in range(N):
    x,y = X[i],Y[i]
    indx = x%A
    indy = y%B
    T[indx][indy].append((x,y))
for _ in range(M):
    p,q = map(int,input().split())
    if len(T[p][q])==0:
        print(-1)
    else:
        print(*T[p][q][-1])

In [ ]:
N = int(input())
M = int(input())
A = [0]+list(map(int,input().split()))
C = {i:0 for i in range(1,N+1)}
for j in range(1,M+1):
    B = [0]+list(map(int,input().split()))
    t = A[j]
    cnt = 0
    for i in range(1,N+1):
        if B[i] == t:
            C[i] += 1
            cnt += 1
    C[t] += N-cnt
for i in range(1,N+1):
    print(C[i])
    

In [ ]:
!pip install sortedcontainers

In [ ]:
from sortedcontainers import SortedList
INFTY = 10**10
N = int(input())
A = SortedList()
A.add(-INFTY)
A.add(INFTY)
C = {}
C[0] = []
B = []
for i in range(N):
    c,x = map(int,input().split())
    B.append([c,x,i])
    if c not in C:
        C[c] = []
    C[c].append(x)
    A.add(x)
B = sorted(B,key=lambda x:x[0])
c0 = 0
for i in range(N):
    c,x,_ = B[i]
    if c!=c0:
        for y in C[c0]:
            A.add(y)
        for y in C[c]:
            A.remove(y)
        c0 = c
    ind = A.bisect_right(x)
    d = A[ind]-x
    d = min(d,x-A[ind-1])
    B[i].append(d)
B = sorted(B,key=lambda x:x[2])
for i in range(N):
    print(B[i][3])

In [2]:
N = int(input())
A = set()
for i in range(1,N+1):
    if i*i>N:break
    if N%i==0:
        A.add(i)
        A.add(N//i)
print(N-len(A))

6


In [6]:
N,K = map(int,input().split())
A = [0]+list(map(int,input().split()))
S = [0]*(N+1)
B = set([0])
ans = "No"
for i in range(1,N+1):
    S[i] = S[i-1]^A[i]
    if S[i]^K in B:
        ans = "Yes"
        break
    B.add(S[i])
print(ans)

No


In [ ]:
import heapq
N,M = map(int,input().split())
G = [[[] for _ in range(5)] for _ in range(N+1)]
for _ in range(M):
    u,v = map(int,input().split())
    for j in range(5):
        G[u][j].append((v,0))
        G[v][j].append((u,0))
        if j<4:
            G[u][j].append((v,j+1))
            G[v][j].append((u,j+1))
K = int(input())
B = [0]*(N+1)
if K>0:
    A = list(map(int,input().split()))
    for i in range(K):
        B[A[i]] = 1
INFTY = 10**6
dist = [[INFTY for _ in range(5)] for _ in range(N+1)]
dist[1][0] = 0
que = [(0,1,0)]  # (距離,頂点)
while que:
    d,v,j = heapq.heappop(que)
    if dist[v][j]<d:continue
    for y,c in G[v][j]:
        if B[y]==1 and c==j+1 and dist[y][j+1]>d+1:
            dist[y][j+1] = d+1
            heapq.heappush(que,(d+1,y,j+1))
        elif B[y]==0 and c==0 and dist[y][0]>d+1:
            dist[y][0] = d+1
            heapq.heappush(que,(d+1,y,0))
ans = min(dist[N][j] for j in range(5))
if ans>=INFTY:
    print(-1)
else:
    print(ans)

2


In [2]:
import sys
sys.setrecursionlimit(10**7)
N = int(input())
memo = {}
def f(x):
    if x==1:
        return ""
    if x in memo:
        return memo[x]
    if x%2==0:
        k = 0
        y = x
        while y%2==0:
            y //= 2
            k += 1
        memo[x] = "/"*k+f(y)
        return memo[x]
    else:
        if (3*x+1)%4==0:
            memo[x] = "+"+"/"*2+f((3*x+1)//4)
            return memo[x]
        else:
            memo[x] = "-"+"/"*2+f((3*x-1)//4)
            return memo[x]
ans = f(N)
print(len(ans))
print(ans)

100
+//-//+///-//-///+//+/////+//-//-//+//-///-//-//-///+//-////+///+//-//////+/////-///+//+////-//+////


In [4]:
N,K = map(int,input().split())
A = [int(input()) for _ in range(N)]
ans = 0
for i in range(1<<N):
    tot = 0
    for j in range(N):
        if i>>j & 1:
            tot += A[j]
    if tot<=K:
        ans = max(ans,tot)
print(ans)

4


In [16]:
N,K = map(int,input().split())
A = [0]+list(map(int,input().split()))+[0]
l0 = 0
for i in range(K-1,0,-1):
    if A[i]==0:
        l0 = i
        break
l1 = l0
for i in range(K-1,l0,-1):
    if A[i]==1:
        l1 = i
        break
left_sum = sum(A[l1:K])
r0 = N+1
for i in range(K+1,N+1):
    if A[i]==0:
        r0 = i
        break
r1 = r0
for i in range(K+1,r0):
    if A[i]==1:
        r1 = i
        break
right_sum = sum(A[K+1:r1])
if r1<r0:
    right_sum += 1
ans = A[K]
if A[K]==1:
    ans += max(left_sum,right_sum)
elif A[K]>1:
    ans += left_sum+right_sum
print(ans)

1


In [17]:
m = int(input())
for _ in range(m):
    ans = []
    a = int(input())
    while a>0:
        if a%2==0:
            ans.append("R")
            a = a//2-1
        else:
            ans.append("L")
            a = a//2
    ans.reverse()
    print("".join(ans))

LRRL
RRR


In [20]:
N,M = map(int,input().split())
A = [list(map(int,input().split())) for _ in range(M)]
B = [0]*(N+1)
cnt = 0
for i in range(M):
    a,b = A[i]
    if B[a]==0 and b>0:
        cnt += 1
        B[a] = b
    elif B[a]>0 and b==0:
        cnt -= 1
        B[a] = 0
    elif B[a]>0 and b>0:
        B[a] = b
    if cnt==N:
        break
print(sum(B))

15


In [2]:
N = int(input())
a = int(input())
s = input().strip()
b = int(input())
A = list(range(1,N+1))
cur = A.index(a)
ans = []
if s=="CW":
    while A:
        A.remove(a)
        ans.append(a)
        if len(A)==0:break
        m = b-1
        if cur>=len(A):
            cur = 0
        while m>0:
            if cur+1<len(A):
                cur += 1
            else:
                cur = 0
            m -= 1
        a = A[cur]
else:
    while A:
        A.remove(a)
        ans.append(a)
        if len(A)==0:break
        m = b-1
        cur -= 1
        if cur<0:
            cur = len(A)-1
        while m>0:
            if cur-1>=0:
                cur -= 1
            else:
                cur = len(A)-1
            m -= 1
        a = A[cur]
print(*ans)

ValueError: invalid literal for int() with base 10: ''

In [3]:
N = int(input())
cnt = 0
for i in range(N):
    cnt += i
print(cnt)


45


In [1]:
y = 5/3
print(y)

1.6666666666666667


In [ ]:
3+4